In [1]:

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "docs/CV_Arktic_D.F.S..pdf"

loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

3


In [4]:

print(docs[0].page_content[0:100])
print(docs[0].metadata)

EXPERIENCIA PROFESIONAL 
DevOps Openshift Engineer 
Consulting | Dic 2024 – Currently 
 
- Actualmen
{'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2025-02-19T14:10:00+01:00', 'author': 'Alberto Quesada Leon', 'moddate': '2025-02-19T14:10:00+01:00', 'source': 'docs/CV_Arktic_D.F.S..pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}


In [12]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    """You are an assistant for question-answering tasks.
    You are a technical expert in personnel selection
    Use the following pieces of retrieved context to answer 
    the question. If you don't know the answer, say that you 
    don't know. Use three sentences maximum and keep the 
    answer concise.
    \n\n
    {context}"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What is this article about?"})

results["answer"]

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


'The article outlines the academic and professional background of an individual specializing in systems administration and container orchestration, particularly with OpenShift and related technologies. It details their academic qualifications, certifications, work experience, and technical skills, highlighting expertise in managing cloud infrastructure and automation processes. Additionally, it emphasizes their proficiency in tools like AWS, GitOps, and several Red Hat certifications.'

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
